In [23]:
# ------------------------------ Install Chroma and Google Gen AI ------------------------------
#!pip install chromadb
#!pip install google-generativeai

In [24]:
# ------------------------------ Setup ------------------------------

import os
import chromadb.utils.embedding_functions as embedding_functions
import chromadb
import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")

genai.configure(api_key=GOOGLE_API_KEY)

google_ef = embedding_functions.GoogleGenerativeAiEmbeddingFunction(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel("gemini-pro")

In [40]:
# ------------------------------ Functions ------------------------------

for m in genai.list_models():
    if "embedContent" in m.supported_generation_methods:
        print(m.name)

def build_prompt(query, context):
    base_prompt = {
        "content": "I am going to ask you a question, which I would like you to answer"
        " based only on the provided context, and not any other information."
        " If there is not enough information in the context to answer the question,"
        ' say "I am not sure", then try to make a guess.'
        " If there is enough information, do not mention the word context",
    }
    user_prompt = {
        "content": f" The question is '{query}'. Here is all the context you have:"
        f'{(" ").join(context)}',
    }

    # combine the prompts to output a single prompt string
    system = f"{base_prompt['content']} {user_prompt['content']}"

    return system


def get_gemini_response(query, context):
    response = model.generate_content(build_prompt(query, context))
    return response.text



models/embedding-001


In [ ]:
# ------------------------------ Create Chroma Client and Collection ------------------------------


# Create a client
client = chromadb.Client()

nokia_collection = client.get_or_create_collection(name="nokia", embedding_function=google_ef)


# Add documents
nokia_collection.add(
    documents=["Nokia on Thursday said that it will begin a two-year 600 million euro ($653 million) share buyback this quarter, after reporting that its profit plunged in 2023.",
               "Nokia shares were 7% higher at around 8.19 a.m. London time on Thursday.",
               "One of the world’s largest mobile network equipment makers, Nokia posted fourth-quarter net sales of 5.7 billion euros, a 23% year-on-year decline. Comparable operating profit fell 27% year-on-year to 846 million.",
               "“In 2023 we saw a meaningful shift in customer behavior impacting our industry driven by the macro-economic environment and high interest rates along with customer inventory digestion,” Nokia CEO Pekka Lundmark said in a statement."],
    metadatas=[{"source": "my_source"}, {"source": "my_source"}, {"source": "my_source"}, {"source": "my_source"}],
    ids=["id1", "id2", "id3", "id4"],
)

# Query
nokia_collection = client.get_collection(name="nokia", embedding_function=google_ef)


In [53]:
# ------------------------------ Query and results ------------------------------

# Successful querys
# query = "How much will be Nokia's share buyback?"
# query = "What were the reasons for Nokia shares increase on Thursday?"

# Unsuccessful querys
# query = "What was Nokia shares price increase on Thursday?"
# query = "Can you infer from all the documents what the reason for Nokia's year-on-year decline in net sales and operating profit was in 2023?"

query = "How much was the share buyback in dollars?"

# Query the collection to get the 2 most relevant results
results = nokia_collection.query(query_texts=[query], n_results=1, include=["documents", "metadatas"])


# Get the response from Gemini
response = get_gemini_response(query, results["documents"][0])

# Output, with sources
print(response)


653 million
